1. Importar Bibliotecas e Configurar Ambiente

In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Configurações globais
import warnings
warnings.filterwarnings("ignore")


2. Carregar os Dados

In [40]:
# Carregar os datasets
train_path = r'D:\Documents\Estudos\Projetos\MACHINE_LEARNING\kaggle_titanic_machine_learning_from_disaster\data\raw\train.csv'
test_path = r'D:\Documents\Estudos\Projetos\MACHINE_LEARNING\kaggle_titanic_machine_learning_from_disaster\data\raw\test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Visualizar os dados
print("Treinamento:")
print(train_data.head())
print("\nTeste:")
print(test_data.head())


Treinamento:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN 

3. Limpeza e Engenharia de Atributos

In [41]:
def preprocess_data(data):
    data = data.copy()

    # Preencher valores ausentes
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['Fare'].fillna(data['Fare'].median(), inplace=True)
    data['Embarked'].fillna('S', inplace=True)
    
    # Codificar variáveis categóricas
    data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
    data['Embarked'] = data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
    
    # Criar novos atributos
    data['FamilySize'] = data['SibSp'] + data['Parch']
    data['IsAlone'] = (data['FamilySize'] == 0).astype(int)
    
    # Selecionar atributos úteis
    features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'IsAlone']
    return data[features]

# Pré-processar dados de treinamento e teste
X_train = preprocess_data(train_data)
y_train = train_data['Survived']
X_test = preprocess_data(test_data)


4. Validação Cruzada e Grid Search

In [42]:
def evaluate_model(model, param_grid, X, y, cv=5):
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='accuracy',
        cv=cv,
        verbose=1,
        n_jobs=-1
    )
    grid_search.fit(X, y)
    return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_


5. Modelos e Hiperparâmetros

In [43]:
# Modelos e hiperparâmetros
models = {
    'Random Forest': (RandomForestClassifier(random_state=42), {
        'n_estimators': [100, 200],
        'max_depth': [5, 10],
        'min_samples_split': [2, 5]
    }),
    'XGBoost': (XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), {
        'n_estimators': [100, 150],
        'max_depth': [3, 5],
        'learning_rate': [0.05, 0.1]
    }),
    'LightGBM': (LGBMClassifier(random_state=42), {
        'n_estimators': [100, 150],
        'max_depth': [3, 5],
        'learning_rate': [0.05, 0.1]
    }),
    'CatBoost': (CatBoostClassifier(verbose=0, random_state=42), {
        'iterations': [100, 150],
        'depth': [3, 5],
        'learning_rate': [0.05, 0.1]
    }),
    'Logistic Regression': (LogisticRegression(), {
        'C': [0.1, 1, 10],
        'solver': ['liblinear']
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    }),
    'SVM': (SVC(random_state=42), {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    })
}


6. Testar os Modelos

In [44]:
# Avaliar os modelos
results = []
for name, (model, param_grid) in models.items():
    print(f"Treinando: {name}")
    best_model, best_params, best_score = evaluate_model(model, param_grid, X_train, y_train)
    results.append({
        'Model': name,
        'Best Params': best_params,
        'Best Score': best_score
    })

# Resultados
results_df = pd.DataFrame(results)
results_df.sort_values(by='Best Score', ascending=False, inplace=True)
print(results_df)


Treinando: Random Forest
Fitting 5 folds for each of 8 candidates, totalling 40 fits


Treinando: XGBoost


AttributeError: 'super' object has no attribute '__sklearn_tags__'

7. Previsões e Submissão

In [ ]:
# Selecionar o melhor modelo
best_model_name = results_df.iloc[0]['Model']
print(f"Melhor Modelo: {best_model_name}")

# Treinar no conjunto completo e prever no teste
best_model = models[best_model_name][0]
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Gerar o arquivo de submissão
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': y_pred
})


In [ ]:
submission.to_csv('data/submission.csv', index=False)
print("Arquivo de submissão gerado: data/submission.csv")
